### Data Cleaning + Analysis Notebook 4

***NOTE***: 

- Datset Aquired from PolicyMap / Census: Decennial Census and American Community Survey (ACS)

- Handling missing values, aligning variable definitions across data sources, and performing initial exploratory analysis to verify data consistency and identify early trends.

In [2]:
import pandas as pd

# Load dataset
income = pd.read_csv("Data_Files_Uncleaned/Household_Median_Income_2019_2023.csv")


In [ ]:
# Clean column names 
income.columns = (
    income.columns.str.strip()
    .str.replace(" ", "_")
    .str.replace(r"[^\w\s]", "", regex=True)
)

# Keep only relevant columns
income = income[
    [
        "Geography_Type_Description",
        "Geography_Name",
        "Sits_in_State",
        "GeoID",
        "Median_Household_Income",
        "Data_Time_Period",
    ]
]


In [ ]:
# Convert numeric column
income["Median_Household_Income"] = pd.to_numeric(income["Median_Household_Income"], errors="coerce")

# Drop missing or invalid entries
income = income.dropna(subset=["Median_Household_Income"])

# Verify
print(income.info())
print(income.head())


<class 'pandas.core.frame.DataFrame'>
Index: 3220 entries, 0 to 3231
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Geography_Type_Description  3220 non-null   object 
 1   Geography_Name              3220 non-null   object 
 2   Sits_in_State               3220 non-null   object 
 3   GeoID                       3220 non-null   int64  
 4   Median_Household_Income     3220 non-null   float64
 5   Data_Time_Period            3220 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 176.1+ KB
None
  Geography_Type_Description Geography_Name Sits_in_State  GeoID  \
0                     County        Autauga            AL   1001   
1                     County        Baldwin            AL   1003   
2                     County        Barbour            AL   1005   
3                     County           Bibb            AL   1007   
4                     County      

In [5]:
# Save cleaned dataset
income.to_csv("us_household_median_income_cleaned.csv", index=False)

print("Cleaned dataset saved as 'us_household_median_income_cleaned.csv'")


Cleaned dataset saved as 'us_household_median_income_cleaned.csv'


***Data Analysis (validating data integrity)***: 

- Conducting Data Analysis below to determine if we gain meaningful insights from the data

In [ ]:
# Load datasets
insurance = pd.read_csv("Data_Files_Cleaned/insurance_and_population_merged_2019_2023_us_cleaned.csv")

# Clean income dataset
income = income.rename(columns={
    "Geography_Name": "County",
    "Sits_in_State": "State",
    "Median_Household_Income": "Median_Household_Income"
})[["County", "State", "Median_Household_Income"]]

income["Median_Household_Income"] = pd.to_numeric(income["Median_Household_Income"], errors="coerce")

# Use only existing insurance columns
insurance = insurance.rename(columns=str.strip)

# Merge datasets
merged = pd.merge(income, insurance, on=["County", "State"], how="inner")

# Drop missing income or population rows
merged = merged.dropna(subset=["Median_Household_Income", "Total_Population"])
merged = merged[merged["Total_Population"] > 0]

merged.head()


,County,State,Median_Household_Income,GeoID,Total_Population,Insured,Uninsured,Percent_Insured,Percent_Uninsured
0,Autauga,AL,69841.0,1001,59285.0,53685.0,4268.0,90.554103,7.199123
1,Baldwin,AL,75019.0,1003,239945.0,217366.0,19338.0,90.589927,8.059347
2,Barbour,AL,44290.0,1005,24757.0,19335.0,2350.0,78.099123,9.492265
3,Bibb,AL,51215.0,1007,22152.0,19051.0,1730.0,86.001264,7.809679
4,Blount,AL,61096.0,1009,59292.0,52832.0,5997.0,89.104770,10.114349


In [8]:
# Save merged dataset
merged.to_csv("income_insurance_merged.csv", index=False)

print("Merged dataset saved as 'income_insurance_merged.csv'")


Merged dataset saved as 'income_insurance_merged.csv'


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# Create income quartiles
merged["Income_Quartile"] = pd.qcut(merged["Median_Household_Income"], 4, labels=["Low", "Mid-Low", "Mid-High", "High"])

# Box Plot: Percent Insured by Income Quartile
fig_box = px.box(
    merged,
    x="Income_Quartile",
    y="Percent_Insured",
    color="Income_Quartile",
    title="Percent Insured Distribution by Median Income Quartile (2019–2023)",
    labels={"Income_Quartile": "Median Household Income Quartile", "Percent_Insured": "Percent Insured (%)"},
    points="all",
    template="plotly_white"
)

fig_box.update_layout(
    title_x=0.5,
    showlegend=False
)
fig_box.show()


In [13]:
# Histogram: Distribution of Percent Insured
fig_hist = px.histogram(
    merged,
    x="Percent_Insured",
    nbins=30,
    color_discrete_sequence=["#1f77b4"],
    title="Distribution of Percent Insured Across All US Counties (2019–2023)",
    labels={"Percent_Insured": "Percent Insured (%)"},
    template="plotly_white"
)

fig_hist.update_layout(title_x=0.5)
fig_hist.show()


***NOTE***: 

Box Plot Insight:

- The median percent insured across all income levels remains very high (often above 85–90%).

- While higher-income quartiles may have slightly higher coverage, the gap is minimal — indicating broad access to insurance regardless of income in many counties.

Distribution Insight:

- The histogram confirms that most counties cluster tightly around 85–95% insured, with few outliers below 75%.

- This shows the lack of variance makes it hard to find a strong correlation with income.

---

- Public programs like Medicaid and Affordable Care Act (ACA) subsidies have dramatically increased insurance rates, even in lower-income counties.

- Many low-income, rural states show high insurance rates because of Medicaid expansion, while wealthier urban counties may have a mix of private insurance.

- Thus, income isn’t the sole driver — state-level policies and federal aid participation are major influencing factors.

- Even though insurance coverage is high, differences in healthcare access and utilization (seeing a primary care doctor vs. using the ER) may depend on PCP availability rather than insurance status alone.

- High insurance coverage ≠ high primary care access. Many areas — especially rural or low-physician-density regions — may have insured residents who still rely on ERs for care because PCP access is limited.

In [ ]:
# Choropleth Map: Average Percent Insured by State
state_summary = (
    merged.groupby("State", as_index=False)["Percent_Insured"]
    .mean()
    .rename(columns={"Percent_Insured": "Avg_Percent_Insured"})
)

fig_map = px.choropleth(
    state_summary,
    locations="State",
    locationmode="USA-states",
    color="Avg_Percent_Insured",
    color_continuous_scale="Viridis",
    scope="usa",
    title="Average Percent Insured by State (2019–2023)",
    labels={"Avg_Percent_Insured": "Average % Insured"}
)

fig_map.update_layout(
    title_x=0.5,
    geo=dict(showframe=False, showcoastlines=False),
    template="plotly_white"
)

fig_map.show()
